In [1]:
import requests

In [2]:
# Define the URL of your endpoint
url = 'http://localhost:8080/intent'  # Replace with your actual endpoint URL




In [4]:
# Scenario: Request with valid body and text
payload_valid = {'text': 'flight 444 information'}
response_valid = requests.post(url, json=payload_valid)

print('Status: ',  response_valid.status_code)

'''
expected output:

Status 200

{
 "intents": [{
   "label": "flight"
 }, {
   "label": "aircraft"
 }, {
   "label": "capacity"}]
}
'''
response_valid.json()

Status:  200


{'intents': [{'label': 'flight'},
  {'label': 'flight_no'},
  {'label': 'airline'}]}

In [5]:
# Scenario: Request with missing body
response_missing_body = requests.post(url)

print('Status: ',  response_missing_body.status_code)
# print(response_missing_body.json())


'''
expected output:

STATUS 400

{
 "label": "BODY_MISSING",
 "message": "Request doesn't have a body."
}
'''
response_missing_body.json()

Status:  400


{'detail': {'label': 'BODY_MISSING',
  'message': "Request doesn't have a body."}}

In [41]:
# Request with more fields than necessary gets accepted
payload_more_fields = {'wrong_field': '122', 'text': 'where is my flight'}
response_more_fields = requests.post(url, json=payload_more_fields)
print('Status: ', response_more_fields.status_code)

'''
expected output:

Status 200

{
 "intents": [{
   "label": "flight"
 }, {
   "label": "aircraft"
 }, {
   "label": "capacity"}]
}
'''

response_more_fields.json()

Status:  200


{'intents': [{'label': 'flight'},
  {'label': 'airline'},
  {'label': 'flight_no'}]}

In [42]:
# Requests with body missing 'text' field altogether get Status: 400 as well
payload_missing_field = {'invalid_field': 'value'}
response_missing_field = requests.post(url, json=payload_missing_field)

'''
expected output:

Status: 400

{
 "label": "TEXT_MISSING",
 "message": "\"text\" missing from request body."
}

'''

print('Status: ',  response_missing_field.status_code)

response_missing_field.json()

Status:  400


{'detail': {'label': 'TEXT_MISSING',
  'message': '"text" missing from request body.'}}

In [32]:
# same with empty json
payload_empty_json = {}
response_empty_json = requests.post(url, json=payload_empty_json)

print('Status: ',  response_empty_json.status_code)

'''
expected output:

Status 400

{
 "label": "TEXT_MISSING",
 "message": "\"text\" missing from request body."
}

'''

response_empty_json.json()

Status:  400


{'detail': {'label': 'TEXT_MISSING',
  'message': '"text" missing from request body.'}}

## Request handling 


While the above requests don't trigger any **RequestValidationError**, in some cases the error is triggered causing the server to return unwanted startuses.  

By handling these validation errors, we can configure the server to return  **500 Internal Server Error**.


In [ ]:
# Exception handler for RequestValidationError
''' 
@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request, exc):
    """
    Exception handler for RequestValidationError.

    This method catches RequestValidationError raised during request validation 
    and responds with a 500 Internal Server Error status code instead of the predefined fastapi error for value validation.

    Parameters:
        request (Request): The request object.
        exc (RequestValidationError): The RequestValidationError instance.

    Returns:
        HTTPException: An HTTPException with a 500 status code and error detail.
    
    Example Requests Handled:
        - Missing required fields in the request body.
        - Incorrect data types for fields.
        - Invalid field values failing validation criteria.
        - Incorrect data format in the request.

    """
    raise HTTPException(status_code=500, detail={"label": "INTERNAL_ERROR", "message": str(exc)})
'''

### Scenario: Request with  with non-string value

**without** RequestValidationError custom handling 

In [8]:
# Scenario: Request with non-string value 
payload_invalid = {'text': 3}
response_invalid = requests.post(url, json=payload_invalid)

print('Status: ',  response_invalid.status_code)

response_invalid.json()

Status:  422


{'detail': [{'type': 'string_type',
   'loc': ['body', 'text'],
   'msg': 'Input should be a valid string',
   'input': 3}]}

In [9]:
# Scenario: Request with non-string text
payload_large = {'text': 'A' * 100000}  # Large text for testing performance
response_invalid = requests.post(url, json=payload_large)

print('Status: ',  response_invalid.status_code)


# validation error is handled
response_invalid.json()

Status:  422


{'detail': [{'type': 'string_too_long',
   'loc': ['body', 'text'],
   'msg': 'String should have at most 500 characters',
   'input': '...',
   'ctx': {'max_length': 500}}]}

In [10]:
# Requests with empty string 'text' field value get Status: 500
# since service only accepts non-empty strings until 500 characters
payload_empty_text = {'text': ''}
response_empty_text = requests.post(url, json=payload_empty_text)
print('Status: ',  response_empty_text.status_code)
'''
expected output:

Status 500 

{
 "label": "INTERNAL_ERROR",
 "message": "<ERROR_MESSAGE>"
}
'''

# validation error is handled
response_empty_text.json()

Status:  422


{'detail': [{'type': 'string_too_short',
   'loc': ['body', 'text'],
   'msg': 'String should have at least 1 character',
   'input': '',
   'ctx': {'min_length': 1}}]}

**with** RequestValidationError custum handling 

In [31]:
# Scenario: Request with non-string text
payload_invalid = {'text': 3}
response_invalid = requests.post(url, json=payload_invalid)

print('Status: ',  response_invalid.status_code)

# validation error c
response_invalid.json()

Status:  500


{'detail': {'label': 'INTERNAL_ERROR',
  'message': "[{'type': 'string_type', 'loc': ('body', 'text'), 'msg': 'Input should be a valid string', 'input': 3}]"}}

In [44]:
# Scenario: Request with non-string text
payload_large = {'text': 'A' * 100000}  # Large text for testing performance
response_invalid = requests.post(url, json=payload_large)

print('Status: ',  response_invalid.status_code)


# validation error is handled
response_invalid.json()

Status:  500


{'detail': {'label': 'INTERNAL_ERROR',
  'message': "[{'type': 'string_too_long', 'loc': ('body', 'text'), 'msg': 'String should have at most 500 characters', 'input': '...', 'ctx': {'max_length': 500}}]"}}

In [45]:
# Requests with empty string 'text' field value get Status: 500
# since service only accepts non-empty strings until 500 characters
payload_empty_text = {'text': ''}
response_empty_text = requests.post(url, json=payload_empty_text)
print('Status: ',  response_empty_text.status_code)
'''
expected output:

Status 500 

{
 "label": "INTERNAL_ERROR",
 "message": "<ERROR_MESSAGE>"
}
'''

# validation error is handled
response_empty_text.json()

Status:  500


{'detail': {'label': 'INTERNAL_ERROR',
  'message': "[{'type': 'string_too_short', 'loc': ('body', 'text'), 'msg': 'String should have at least 1 character', 'input': '', 'ctx': {'min_length': 1}}]"}}

## More Edge cases

In [46]:
# Scenario: Request with text set to None get Status 400 
payload_invalid = {'text': None}
response_invalid = requests.post(url, json=payload_invalid)

print('Status: ',  response_invalid.status_code)

'''
expected output:

Status 400 

{
 "label": "TEXT_MISSING",
 "message": "\"text\" missing from request body."
}
'''

response_invalid.json()

Status:  400


{'detail': {'label': 'TEXT_MISSING',
  'message': '"text" missing from request body.'}}

In [29]:
# Scenario: Request with non-string text
payload_special_characters = {'text': '@@Flight $444: !@#$%^&*()'}
response_invalid = requests.post(url, json=payload_special_characters)

print('Status: ',  response_invalid.status_code)

response_invalid.json()

Status:  200


{'intents': [{'label': 'flight_no'},
  {'label': 'flight'},
  {'label': 'airline'}]}

In [27]:
# Scenario: Request with non-string text
payload_unicode = {'text': 'flight 444: \u00e9'}
response_invalid = requests.post(url, json=payload_unicode)

print('Status: ',  response_invalid.status_code)

response_invalid.json()

Status:  200


{'intents': [{'label': 'flight'},
  {'label': 'flight_no'},
  {'label': 'airport'}]}

In [25]:
# Scenario: Request with non-string text
payload_whitespace = {'text': '   Text with leading and trailing whitespace   '}
response_invalid = requests.post(url, json=payload_whitespace)

print('Status: ',  response_invalid.status_code)

response_invalid.json()

Status:  200


{'intents': [{'label': 'flight'},
  {'label': 'flight_time'},
  {'label': 'airfare'}]}

## Testing Valid Requests

In [24]:
# Scenario: Request with valid body and text
payload_valid = {'text': 'round trip flights from orlando to montreal please'} # TARGET LABEL: flight
response_valid = requests.post(url, json=payload_valid)

print('Status: ',  response_valid.status_code)

'''
expected output:

Status 200

'''
response_valid.json()

Status:  200


{'intents': [{'label': 'flight'}, {'label': 'city'}, {'label': 'flight_no'}]}

In [23]:
# Scenario: Request with valid body and text
payload_valid = {'text': 'show me ground transportation in phoenix'} # TARGET LABEL: ground_service
response_valid = requests.post(url, json=payload_valid)

print('Status: ',  response_valid.status_code)

'''
expected output:

Status 200

'''
response_valid.json()

Status:  200


{'intents': [{'label': 'ground_service'},
  {'label': 'airport'},
  {'label': 'city'}]}